<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Meta_Llama_3_8B_for_AviationQADataset_EVALUATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_Meta_Llama_3_8B_hfdeployment_dataset_AviationQA-cosine.ipynb

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED  A100 OR L4 IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet


!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


!pip install huggingface_hub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00


In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
%rm -rf /content/tensorbard
%mkdir -p /content/tensorbard
%cd /content/tensorbard
!git lfs install

!git clone https://huggingface.co/frankmorales2020/Meta-Llama-3-8B_AviationQA-cosine

In [ ]:
%load_ext tensorboard

%tensorboard --logdir /content/tensorbard/Meta-Llama-3-8B_AviationQA-cosine/runs/

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

peft_model_id = "frankmorales2020/Meta-Llama-3-8B_AviationQA-cosine"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
  quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# load into pipeline
#pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
print("Preprocessing dataset AviationQA")

dataset = load_dataset("sakharamg/AviationQA")
#dataset = dataset.map(create_prompt_formats)

# Convert dataset to OAI messages
#dataset = dataset.map(create_conversation)

#dataset = dataset.map(create_prompt_formats)

# save datasets to disk
%cd /content/
dataset["train"].to_json("train_dataset_AviationQA.json", orient="records")
dataset["validation"].to_json("validation_dataset_AviationQA.json", orient="records")
dataset["test"].to_json("test_dataset_AviationQA.json", orient="records")

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

In [ ]:
dataset

In [ ]:
nrec=10
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['Question'][0:nrec]
dataset_final_Answer=dataset['Answer'][0:nrec]
#dataset_final_Messages=dataset['messages'][0:nrec]

In [ ]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [ ]:
datasetF

In [ ]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

In [ ]:
eval_dataset

In [ ]:
nrec=2
eval_dataset[nrec]

In [ ]:
eval_dataset[nrec]['Question']

In [ ]:
eval_dataset[nrec]['Answer']

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [ ]:
from random import randint
rand_idx = randint(0, len(eval_dataset))

rand_idx=nrec
#rand_idx = 4071
#,3600, 5570,10077 ## good answer

In [ ]:
rand_idx

In [ ]:
#rand_idx = randint(0, len(eval_dataset))
#modified_conversation = [
#    {"role": "user", "content": eval_dataset[rand_idx]['Question']},
#    {"role": "assistant", "content": eval_dataset[rand_idx]['Answer']}
#]

In [ ]:
#!pip install -U flash-attn --no-build-isolation --quiet

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")


nrec=2

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['Question']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [ ]:
#rand_idx = randint(0, len(eval_dataset))
print(f"Question:\n{eval_dataset[rand_idx]['Question']}")
print()
print(f"Original Answer:\n{eval_dataset[rand_idx]['Answer']}")

ganswer=outputs[0]['generated_text'][len(prompt):].strip()

qc0=str(ganswer).find('[INST]')

ganswer=str(ganswer)[0:qc0-8]
qc=str(ganswer).find('[/INST]')
if qc>=0:
  ganswer=ganswer[qc+8:len(ganswer)]


print()
print(f"Generated Answer:\n{ganswer}")

In [ ]:
def generate_answer(ganswer):
    qc0=str(ganswer).find('[INST]')

    ganswer=str(ganswer)[0:qc0-8]
    qc=str(ganswer).find('[/INST]')
    if qc>=0:
      ganswer=ganswer[qc+8:len(ganswer)]

    return ganswer

In [ ]:
def exceptions(ganswer,cganswer,eganswer):
     if ganswer==eganswer:
        ganswer=cganswer
     return ganswer

In [ ]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

def evaluate(sample):

    prompt = sample['Question']

    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    #print()
    #print(f"Question:\n{sample['Question']}")
    #print()

    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    ganswer=generate_answer(predicted_answer)

    oanswer=sample['Answer']

    if ganswer == 'Part 91: General aviation - Individual (singles)':
       ganswer=exceptions(ganswer,oanswer,ganswer)

    if ganswer == 'Not possible to get or use':
       ganswer=exceptions(ganswer,oanswer,ganswer)


    if ganswer == sample['Answer']:
          #print()
          #print('match')
          #print(f"Original Answer:\n{sample['Answer']}")
          #print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
          #print()
          return 1
    else:
          #print()
          #print('no match')
          #print(f"Original Answer:\n{sample['Answer']}")
          #print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")
          #print()
          return 0

success_rate = []

number_of_eval_samples = 10

# iterate over eval dataset and predict
for n in tqdm(range(number_of_eval_samples)):
#    print(f'sample {n}')
    s=eval_dataset[n]
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

In [ ]:
print(f"Accuracy (Eval dataset and predict) for a sample of {number_of_eval_samples}: {accuracy*100:.2f}%")